In [27]:
# paquetes
import pandas as pd
import random
import math
import copy
import matplotlib.pyplot as plt

In [29]:
# 1. Carga de datos
df = pd.read_csv("data_reg_simbolica.tsv", sep="\t")
# limpiamos columnas y extraemos X, Y (por posición)
df.columns = [col.strip().lstrip('#') for col in df.columns]
X = df.iloc[:,0].values.tolist()
Y = df.iloc[:,1].values.tolist()

In [30]:
# 2. Definición de nodos GP
class Node:
    def __init__(self, name, hijos=None):
        self.name = name
        self.hijos = hijos or []

    def eval(self, x_val):
        # terminal: variable x o constante
        if not self.hijos:
            if self.name == 'x':
                return x_val
            try:
                return float(self.name)
            except ValueError:
                return 0.0
        # recursión
        vals = [c.eval(x_val) for c in self.hijos]
        # primitiva binaria
        if self.name == 'ADD':
            return vals[0] + vals[1]
        if self.name == 'SUB':
            return vals[0] - vals[1]
        if self.name == 'MUL':
            return vals[0] * vals[1]
        if self.name == 'DIV':
            try:
                return vals[0] / vals[1]
            except ZeroDivisionError:
                return 1.0
        if self.name == 'POW':
            try:
                r = math.pow(vals[0], vals[1])
                return r if math.isfinite(r) else 1.0
            except Exception:
                return 1.0
        # primitiva unaria
        if self.name == 'SIN':
            try:
                return math.sin(vals[0])
            except Exception:
                return 0.0
        if self.name == 'COS':
            try:
                return math.cos(vals[0])
            except Exception:
                return 0.0
        if self.name == 'EXP':
            try:
                return math.exp(vals[0])
            except Exception:
                return float('inf')
        if self.name == 'LOG':
            try:
                return math.log(abs(vals[0]))
            except Exception:
                return 0.0
        if self.name == 'SQRT':
            return math.sqrt(abs(vals[0]))
        if self.name == 'NEG':
            return -vals[0]
        # desconocido
        return 0.0

    def lectura(self):
        if not self.hijos:
            return self.name
        # unarios
        if self.name in ('SIN','COS','EXP','LOG','SQRT','NEG'):
            c = self.hijos[0].lectura()
            op = '-' if self.name=='NEG' else self.name.lower()
            return f"{op}({c})"
        # binarios
        l = self.hijos[0].lectura()
        r = self.hijos[1].lectura()
        op_sym = {'ADD':'+','SUB':'-','MUL':'*','DIV':'/','POW':'^'}[self.name]
        return f"({l} {op_sym} {r})"

In [31]:
# 3. Primitivas y terminales
FUNCTIONS = [ ('ADD',2), ('SUB',2), ('MUL',2), ('DIV',2), ('POW',2),
              ('SIN',1), ('COS',1), ('EXP',1), ('LOG',1), ('SQRT',1), ('NEG',1) ]
TERMINALS = ['x']
# constantes efímeras generadas en [-10,10]

In [32]:
# 4. Generación de árboles (full/grow)
def gen_tree(max_depth):
    def grow(depth):
        if depth>=max_depth or (depth>0 and random.random()<0.3):
            # terminal: x o constante
            if random.random()<0.5:
                return Node('x')
            return Node(str(round(random.uniform(-10,10),2)))
        name, arity = random.choice(FUNCTIONS)
        return Node(name, [ grow(depth+1) for _ in range(arity) ])
    return grow(0)

In [33]:
# 5. Cruce y mutación

def all_nodes(node):
    lst = [(None,None,node)]
    for i,ch in enumerate(node.hijos):
        for p,idx,n in all_nodes(ch):
            parent = node if p is None else p
            pos = i if p is None else idx
            lst.append((parent,pos,n))
    return lst


def crossover(t1, t2):
    n1 = copy.deepcopy(t1); n2 = copy.deepcopy(t2)
    nodes1 = all_nodes(n1); nodes2 = all_nodes(n2)
    p1,i1,s1 = random.choice(nodes1)
    p2,i2,s2 = random.choice(nodes2)
    if p1 is None:
        c1 = s2
    else:
        p1.hijos[i1] = s2; c1 = n1
    if p2 is None:
        c2 = s1
    else:
        p2.hijos[i2] = s1; c2 = n2
    return c1,c2


def mutate(tree, max_depth):
    t = copy.deepcopy(tree)
    nodes = all_nodes(t)
    p,i,_ = random.choice(nodes)
    sub = gen_tree(max_depth)
    if p is None:
        return sub
    p.hijos[i] = sub
    return t

In [34]:
# 6. Fitness (MSE)
def fitness(ind):
    errors = []
    for x,y in zip(X,Y):
        pred = ind.eval(x)
        try:
            errors.append((pred-y)**2)
        except Exception:
            errors.append(float('inf'))
    return sum(errors)/len(errors)

In [35]:
# 7. Selección torneo
def torneo(pop, k=3):
    return min(random.sample(pop,k), key=lambda ind: ind_fit[ind])

In [36]:
# 8. Bucle evolutivo
def run_gp(ngen=40, pop_size=200, cxpb=0.5, mutpb=0.2, max_depth=5):
    pop = [gen_tree(max_depth) for _ in range(pop_size)]
    # precomputar fitness
    for ind in pop:
        ind_fit[ind] = fitness(ind)
    top = min(pop, key=lambda i: ind_fit[i])
    no_improve=0
    for gen in range(1,ngen+1):
        new_pop=[]
        while len(new_pop)<pop_size:
            p1,p2 = torneo(pop), torneo(pop)
            if random.random()<cxpb:
                c1,c2 = crossover(p1,p2)
            else:
                c1,c2 = copy.deepcopy(p1), copy.deepcopy(p2)
            if random.random()<mutpb:
                c1 = mutate(c1, max_depth)
            if random.random()<mutpb:
                c2 = mutate(c2, max_depth)
            new_pop.extend([c1,c2])
        pop = new_pop
        # actualizar fitness
        for ind in pop:
            ind_fit[ind] = fitness(ind)
        curr = min(pop, key=lambda i: ind_fit[i])
        if ind_fit[curr] < ind_fit[top]:
            top, no_improve = curr,0
        else:
            no_improve+=1
        print(f"Gen {gen:02d} top MSE={ind_fit[top]:.4f}")
        if no_improve>=10: break

    return top, ind_fit[top]

In [37]:
# 9. Ejecución principal
if __name__ == '__main__':
    # diccionario global de fitness
    ind_fit = {}
    top, top_err = run_gp(ngen=100, pop_size=300, max_depth=10)
    print("\nMejor expresión:", top.lectura())
    print(f"MSE: {top_err:.6f}")


Gen 01 top MSE=4.2837
Gen 02 top MSE=3.6070
Gen 03 top MSE=3.6070
Gen 04 top MSE=3.3898
Gen 05 top MSE=3.3898
Gen 06 top MSE=3.3898
Gen 07 top MSE=3.3759
Gen 08 top MSE=3.2037
Gen 09 top MSE=3.2037
Gen 10 top MSE=3.1373
Gen 11 top MSE=3.1373
Gen 12 top MSE=3.1373
Gen 13 top MSE=3.0582
Gen 14 top MSE=3.0582
Gen 15 top MSE=3.0372
Gen 16 top MSE=2.8274
Gen 17 top MSE=2.8274
Gen 18 top MSE=2.7961
Gen 19 top MSE=2.7961
Gen 20 top MSE=2.7961
Gen 21 top MSE=2.7538
Gen 22 top MSE=2.6579
Gen 23 top MSE=2.6390
Gen 24 top MSE=2.6390
Gen 25 top MSE=2.6390
Gen 26 top MSE=2.4747
Gen 27 top MSE=2.4747
Gen 28 top MSE=2.4747
Gen 29 top MSE=2.4747
Gen 30 top MSE=2.4747
Gen 31 top MSE=2.4747
Gen 32 top MSE=2.1328
Gen 33 top MSE=2.1328
Gen 34 top MSE=2.1328
Gen 35 top MSE=2.1328
Gen 36 top MSE=2.1328
Gen 37 top MSE=2.1328
Gen 38 top MSE=2.1326
Gen 39 top MSE=2.1258
Gen 40 top MSE=2.1258
Gen 41 top MSE=1.9849
Gen 42 top MSE=1.9754
Gen 43 top MSE=1.9751
Gen 44 top MSE=1.9751
Gen 45 top MSE=1.9392
Gen 46 top